In [15]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime as dt
from datetime import timedelta as delta
import numpy as np
import seaborn as sb
import os

In [3]:
signals = pd.DataFrame({
    '10_cross_30':[0,0,1,1,1],
    'MACD_Signal_MACD':[1,1,1,0,0],
    'MACD_lim':[0,0,0,1,1],
    'abv_50':[1,1,1,0,0],
    'abv_200':[0,1,0,0,1],
    'strategy': [1,2,3,4,5],
})
print(signals)

   10_cross_30  MACD_Signal_MACD  MACD_lim  abv_50  abv_200  strategy
0            0                 1         0       1        0         1
1            0                 1         0       1        1         2
2            1                 1         0       1        0         3
3            1                 0         1       0        0         4
4            1                 0         1       0        1         5


In [19]:
def add_signal_indicators(df):
    df['SMA_10'] = ta.sma(df['Adj Close'],length=10)
    df['SMA_30'] = ta.sma(df['Adj Close'],length=30)
    df['SMA_50'] = ta.sma(df['Adj Close'],length=50)
    df['SMA_200'] = ta.sma(df['Adj Close'],length=200)
    
    macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)
    df['MACD'] = macd['MACD_12_26_9']
    df['MACD_signal'] = macd['MACDs_12_26_9']
    df['MACD_hist'] = macd['MACDh_12_26_9']

    df['10_cross_30'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0)
    
    df['MACD_Signal_MACD'] = np.where(df['MACD_signal'] < df['MACD'], 1, 0)
    
    df['MACD_lim'] = np.where(df['MACD']>0, 1, 0)
    
    df['abv_50'] = np.where((df['SMA_30']>df['SMA_50'])
                            &(df['SMA_10']>df['SMA_50']), 1, 0)
                            
    df['abv_200'] = np.where((df['SMA_30']>df['SMA_200'])
                            &(df['SMA_10']>df['SMA_200'])
                            &(df['SMA_50']>df['SMA_200']), 1, 0)
    
    return df

nifty_50_stocks = ['EICHERMOT','HEROMOTOCO','NESTLEIND','ONGC',
                   'BAJAJ-AUTO','TATASTEEL','GRASIM',
                   'BRITANNIA','BAJFINANCE','DIVISLAB','M&M',
                   'HINDUNILVR','HDFCBANK','HDFCLIFE','BHARTIARTL','TCS',
                   'LT','DRREDDY','ULTRACEMCO','SUNPHARMA','NTPC',
                   'TATAMOTORS','UPL','SBIN','HINDALCO','ITC','JSWSTEEL',
                   'COALINDIA','RELIANCE','BPCL','LTIM','MARUTI','HCLTECH',
                   'POWERGRID','WIPRO','SBILIFE','AXISBANK',
                   'ADANIPORTS','ICICIBANK','TITAN','BAJAJFINSV','KOTAKBANK',
                   'TATACONSUM','APOLLOHOSP','INFY','ASIANPAINT',
                   'ADANIENT','INDUSINDBK','TECHM','CIPLA']

In [5]:
nifty_50_stocks = ['TECHM','CIPLA']

In [6]:
def backtest_signals(row):
    global TRADES, trade_in_progress, signals, holding_period
    
    if(trade_in_progress):
        _data = TRADES[-1]
        # time to sell after n holding days
        if(row['index']==_data['sell_index']):
            _data['sell_price'] = round(row['Adj Close'],2)
            _data['sell_date'] = str(pd.to_datetime(row['Date']).date())
            _data['returns'] = round((_data['sell_price']-_data['buy_price'])/_data['buy_price']*100, 3)
            TRADES[-1] = _data
            trade_in_progress = False
            
    else:
        _r = pd.DataFrame([row])
        _r = _r.merge(signals, on=list(signals.columns[:-1]), how='inner')
        strategy = _r.shape[0]
        
        if(strategy>0): 
            trade_in_progress = True
            TRADES.append({
                'strategy': _r['strategy'].values[0],
                'buy_date': str(pd.to_datetime(row['Date']).date()),
                'buy_index': row['index'],
                'sell_date': '',
                'sell_index': row['index'] + holding_period,
                'buy_price': round(row['Adj Close'], 2),
                'sell_price': '',
                'returns': 0,
                'stock': row['stock']
            })

In [ ]:
TRADES = []
trade_in_progress = False
holding_period = 14

for i in nifty_50_stocks:
    _df = yf.download(f"{i}.NS", 
                      period='1d', 
                      start='2022-01-01', 
                      end=(dt.now() + delta(1)).strftime('%Y-%m-%d'), 
                      progress=False)

    _df = add_signal_indicators(_df)

    _df['stock'] = i
    _df.reset_index().reset_index().apply(backtest_signals, axis=1)
    print(f'Done performing backtesting for {i}')
    del _df
    trade_in_progress = False

In [ ]:
TRADES[0:40]

In [ ]:
import matplotlib.pyplot as plt
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=False).head(10)
ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=False).head(11).tail(10)

ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=True).head(10)
ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
from yahooquery import Ticker
stock = Ticker("SBIN.NS")
_df = stock.history(interval='1d', 
                    start='2022-01-01', 
                    end=(dt.now() + delta(1)).strftime('%Y-%m-%d'))
print(_df)
#_df = add_signal_indicators(_df)
#print(_df)

In [ ]:
import pandas as pd
from yahooquery import Ticker

def convert_yahooquery_to_yfinance(df_yahooquery):
    # Step 1: Reset the index to make 'Date' a regular column
    df_yahooquery.reset_index(inplace=True)

    # Step 2: Rename columns
    df_yahooquery.rename(columns={
        'date': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'adjclose': 'Adj Close',
        'volume': 'Volume'
    }, inplace=True)

    # Step 3: Convert "Date" to a string in "YYYY-MM-DD" format
    df_yahooquery['Date'] = pd.to_datetime(df_yahooquery['Date']).dt.strftime('%Y-%m-%d')

    # Step 4: Set "Date" as the index
    df_yahooquery.set_index('Date', inplace=True)

    # Step 5: Remove the "symbol" and "dividends" column
    df_yahooquery.drop('symbol', axis=1, inplace=True)
    df_yahooquery.drop('dividends', axis=1, inplace=True)
    return df_yahooquery

# Example usage:
# Assuming you have a DataFrame df_yahooquery from yahooquery.history

stock = Ticker("SBIN.NS")
_df = stock.history(interval='1d', 
                    start='2022-01-01', 
                    end=(dt.now() + delta(1)).strftime('%Y-%m-%d'))
print(_df)
df_yfinance = convert_yahooquery_to_yfinance(_df)
print(df_yfinance)

In [ ]:
import pandas as pd
from yahooquery import Ticker

def get_stock_info(symbol):
    try:
        tickers = Ticker(symbol, validate=True)
        datasi = tickers.get_modules("summaryProfile quoteType")
        dfsi = pd.DataFrame.from_dict(datasi).T
        dataframes = [pd.json_normalize([x for x in dfsi[module] if isinstance(x, dict)]) for
                      module in ['summaryProfile', 'quoteType']]
        print(dataframes)
        dfsi = pd.concat(dataframes, axis=1)
        
        dfsi = dfsi.set_index('symbol')
        
        return dfsi
    except Exception as e:
        #return f"An error occurred: {e}"
        return None

# Example usage:
symbol = 'TSLA'  # Replace with the stock symbol you want to retrieve information for
stock_info = get_stock_info(symbol)
if isinstance(stock_info, pd.DataFrame):
    print(stock_info['industry'].iloc[0])


In [33]:
end = dt.today().strftime('%Y-%m-%d')
start = (dt.now() - delta(365)).strftime('%Y-%m-%d')

print(start)

2022-11-08


import datetime as dt
if end is None:
    end = dt.datetime.today().strftime('%Y-%m-%d')
elif type(end) is str:
    end = dt.datetime.strptime(end, '%Y-%m-%d')
if start is None:
    start = (dt.datetime.now() - dt.timedelta(365)).strftime('%Y-%m-%d')
elif type(start) is str:
    start = dt.datetime.strptime(start, '%Y-%m-%d')